## Importing modules

In [ ]:
import numpy as np
import pydicom
import glob
import matplotlib.pyplot as plt
import cv2
import skimage
from read_roi import read_roi_file
from ipywidgets import interact, IntRangeSlider


## Getting dcm files

In [ ]:
dir_path = '/Users/giuseppefilitto/Pazienti_anonym_sorted/BO11/T2AX'
files = glob.glob(dir_path + '/*.dcm')

## Reading slice

In [ ]:
def rescale(im, max, min):
    return ((im.astype(float) - min) * (1. / (max - min)) * 255.).astype('uint8')


def read_slices(filename):
    name, ext = filename.split('.')

    if ext != 'dcm':
        raise ValueError('Input filename must be a DICOM file')

    slide = pydicom.dcmread(filename).pixel_array

    return slide


# ordering as istance number
z = [float(pydicom.read_file(f, force=True).get(
    "InstanceNumber", "0") - 1) for f in files]
order = np.argsort(z)
files = np.asarray(files)[order]

slice = [read_slices(f) for f in files]

Max = max([x.max() for x in slice])
Min = min([x.min() for x in slice])

slice = [rescale(x, Max, Min) for x in slice]

slice = np.asarray(slice)


### Slice info

In [ ]:

depth, height, width = slice.shape
print(
    f"The image object has the following dimensions: depth:{depth}, height: {height}, width:{width}")

# 2D Exploration

## Plotting Slice

### Random layer plot

In [ ]:
maxval = depth  # Select random layer number
i = np.random.randint(0, maxval)

print(f"Plotting Layer {i} of Image")

plt.figure(figsize=(12, 7), constrained_layout=True)
plt.imshow(slice[i, :, :], cmap='gray')
plt.axis('off')

### Interactive slice plot

In [ ]:
@interact(layer=(0, slice.shape[0] - 1))
def explore_slice(layer):
    plt.figure(figsize=(12, 7), constrained_layout=True)
    plt.imshow(slice[layer, :, :], cmap='gray')
    plt.title(f'Exploring Layer {layer}', fontsize=20)
    plt.axis('off')

    return layer

## ROIS

### Getting ROIs

In [ ]:
roi_path = '/Users/giuseppefilitto/Pazienti_anonym_sorted/BO11/T2ROI'
rois_list = glob.glob(roi_path + '/*.roi')


def _dict(dict_list):
    '''

    useful to get true_dict since roi is {name file : true_dict}.

    '''

    true_dict = []

    for i in dict_list:
        _dict = list(i.values())

        for j in _dict:
            keys = j.keys()
            vals = j.values()

            _dict = {key: val for key, val in zip(keys, vals)}
            true_dict.append(_dict)

    return true_dict

rois = [read_roi_file(roi) for roi in rois_list]
rois = _dict(rois)

# ordering dictionaries by positions and removing rois without x y coords
rois = sorted(rois, key=lambda d: list(d.values())[-1])
rois = list(filter(lambda d: d['type'] != 'composite', rois))

positions = []
xs = []
ys = []
for i in range(len(rois)):
    position = rois[i]['position']
    x = rois[i]['x']
    y = rois[i]['y']

    x.append(x[0])
    y.append(y[0])

    positions.append(position)
    xs.append(x)
    ys.append(y)

### Exploring ROIs

In [ ]:
@interact(layer=(0, slice.shape[0] - 1))
def explore_roi(layer):
    plt.figure(figsize=(12, 7))
    plt.imshow(slice[layer, :, :], cmap='gray')
    if layer in positions:
        plt.plot(xs[layer - positions[0]], ys[layer - positions[0]], color="red",
                 linestyle='dashed', linewidth=1)
    plt.title(f'Exploring Layer {layer}', fontsize=20)
    plt.axis('off')

    return layer


### ROIs masks

In [ ]:

@interact(layer=(positions[0], positions[-1]))
def explore_mask(layer):



    image = slice[layer, :, :].copy()

    pts = np.array([(x, y) for(x, y) in zip(xs[layer - positions[0]], ys[layer - positions[0]])])

    cv2.drawContours(image, [pts], -1, (255, 255, 255), -1)
    cv2.polylines(image, [pts], isClosed=True, color=(255, 255, 255), thickness=1)
    mask = cv2.threshold(image, 254, 255, cv2.THRESH_BINARY)[1]

    fig, ax=plt.subplots(1, 2, figsize=(12, 7), constrained_layout=True)

    ax[1].imshow(mask, cmap="gray")
    ax[1].set_title(" ROI Mask")
    ax[1].axis('off')

    ax[0].imshow(slice[layer, :, :], cmap="gray")
    ax[0].plot(xs[layer - positions[0]],ys[layer - positions[0]] , color="red",
                linestyle='dashed', linewidth=1)
    ax[0].set_title("ROI")
    ax[0].axis('off')

    fig.suptitle(f"Exploring layer: {layer}",  fontsize=20)


### Appling ROIs Mask

In [ ]:

@interact(layer=(positions[0], positions[-1]))
def explore_masked_image(layer):
    
    image = slice[layer, :, :].copy()

    
    pts = np.array([(x, y) for(x, y) in zip(xs[layer - positions[0]], ys[layer - positions[0]])])

    cv2.drawContours(image, [pts], -1, (255, 255, 255), -1)
    cv2.polylines(image, [pts], isClosed=True, color=(255, 255, 255), thickness=1)
    mask = cv2.threshold(image, 254, 255, cv2.THRESH_BINARY)[1]

    masked_img = cv2.bitwise_and(slice[layer, :, :].copy(), image, mask = mask)

    fig, ax = plt.subplots(1, 3, figsize=(12, 7), constrained_layout=True)

    ax[0].imshow(slice[layer, :, :], cmap="gray")
    ax[0].set_title("Original")
    ax[0].axis('off')

    
    ax[1].imshow(mask, cmap="gray")
    ax[1].set_title("ROI Mask")
    ax[1].axis('off')

    ax[2].imshow(masked_img, cmap="gray")
    ax[2].set_title("Applied ROI mask")
    ax[2].axis('off')
    fig.suptitle(f'Exploring layer {layer}', fontsize=20)
        

# Traditional image processing segmentation

### Exploring histogram

In [ ]:
@interact(layer=(0, slice.shape[0] - 1))
def explore_histogram(layer):

    fig, ax=plt.subplots(1, 2, figsize=(12, 7), constrained_layout=True) 

    ax[0].imshow(slice[layer, :, :], cmap="gray")
    ax[0].set_title("Original image")
    ax[0].axis("off")

    ax[1].hist(slice[layer, :, :].ravel(),256, [0,256], color="black")
    ax[1].set_title("Histogram")

    fig.suptitle(f"Exploring layer: {layer}",  fontsize=20)
    plt.show()

### Histogram Global Thresholding

In [ ]:
@interact(layer=(0, slice.shape[0] - 1), threshold=IntRangeSlider(value=[63,255], min=0, max=255, step=1, ) )
def manual_tresh(layer, threshold):

    image = slice[layer, : ,:].copy()


    t_min = threshold[0]
    t_max = threshold[1]
    

    mask = cv2.threshold(image, t_min, t_max, cv2.THRESH_BINARY_INV)[1]




    fig, ax=plt.subplots(1, 3, figsize=(12, 8), constrained_layout=True) 


    ax[0].imshow(slice[layer, :, :], cmap="gray")
    ax[0].set_title("Original image")
    ax[0].axis("off")

    ax[1].hist(slice[layer, :, :].ravel(),256,[0,256], color="black")
    ax[1].set_title("Histogram")
    ax[1].vlines(t_min,0, 10000, color="red", linestyle="dashed")
    ax[1].vlines(t_max,0, 10000, color="red", linestyle="dashed")
    ax[1].text(t_min, 10000, "$T_{min}$")
    ax[1].text(t_max, 10000, "$T_{max}$")
    ax[1].set_yticks([])
    


    ax[2].imshow(mask, cmap="gray")
    ax[2].set_title("Thresholded Image")
    ax[2].axis("off")



    fig.suptitle(f"Exploring layer: {layer}",  fontsize=20)

    plt.show()

### Adaptive Threshold methods

In [ ]:
layer = 14

img = image = slice[layer, : ,:].copy()
blur = cv2.medianBlur(img,ksize=5)

th = cv2.adaptiveThreshold(blur,150,cv2.ADAPTIVE_THRESH_MEAN_C, cv2.THRESH_BINARY_INV,11,2)
th2 = cv2.adaptiveThreshold(blur,150,cv2.ADAPTIVE_THRESH_GAUSSIAN_C, cv2.THRESH_BINARY_INV,11,2)

fig, ax = plt.subplots(1, 3, figsize=(12,8), constrained_layout=True)

ax[0].imshow(img, "gray")
ax[0].axis("off")

ax[1].imshow(th, "gray")
ax[1].axis("off")

ax[2].imshow(th2, "gray")
ax[2].axis("off")

### Otsu

In [ ]:
    
 @interact(layer=(0, slice.shape[0] - 1))   
def otsu_thresholding(layer):   

    img = slice[layer, : ,:].copy()
    gauss = cv2.GaussianBlur(img, (5, 5), 0)

    t_min = 0
    t_max = 255

    thresh_otsu = cv2.threshold(gauss,t_min, t_max,cv2.THRESH_BINARY_INV+cv2.THRESH_OTSU)[1]

    fig, ax = plt.subplots(1, 2, figsize=(12, 8), constrained_layout=True)

    ax[0].imshow(slice[layer, : ,:], cmap="gray")
    ax[0].axis("off")
    ax[0].set_title("Original")


    ax[1].imshow(thresh_otsu, cmap="gray")
    ax[1].axis("off")
    ax[1].set_title(f"Otsu")


### Boxed Masks

In [ ]:

@interact(layer=(0, slice.shape[0] - 1), threshold=IntRangeSlider(value=[63,120], min=0, max=255, step=1, description='Global Threshold:') )
def box_fov(layer, threshold):


    img = slice[layer, : ,:].copy()
    box = np.zeros(img.shape[:2], np.uint8)
    box[150:350, 150:350] = 255

    boxed_original = cv2.bitwise_and(img,img,mask = box)

    t_min = threshold[0]
    t_max = threshold[1]

    th = cv2.threshold(img, t_min, t_max, cv2.THRESH_BINARY_INV)[1]
    masked_th = cv2.bitwise_and(th,th,mask = box)

    gauss = cv2.GaussianBlur(img, (5, 5), 0)
    thresh_otsu = cv2.threshold(gauss,0, 256,cv2.THRESH_BINARY_INV+cv2.THRESH_OTSU)[1]
    masked_otsu = cv2.bitwise_and(thresh_otsu , thresh_otsu, mask = box)

   
    fig, ax = plt.subplots(1, 3, figsize=(12, 8), constrained_layout=True)

    ax[0].imshow(boxed_original, cmap="gray")
    ax[0].axis("off")
    ax[0].set_title("Original")


    ax[1].imshow(masked_th, cmap="gray")
    ax[1].axis("off")
    ax[1].set_title(f"Global Thresh {t_min, t_max}")

    ax[2].imshow(masked_otsu, cmap="gray")
    ax[2].axis("off")
    ax[2].set_title(f"Otsu")

    


In [ ]:
@interact(layer=(positions[0], positions[-1]), threshold=IntRangeSlider(value=[63,120], min=0, max=255, step=1, description='Global Threshold:') )
def comparing_masks(layer, threshold):

    image = slice[layer, : ,:].copy()
    box = np.zeros(image.shape[:2], np.uint8)
    box[150:350, 150:350] = 255


    pts = np.array([(x, y) for(x, y) in zip(xs[layer - positions[0]], ys[layer - positions[0]])])

    cv2.drawContours(image, [pts], -1, (255, 255, 255), -1)
    cv2.polylines(image, [pts], isClosed=True, color=(255, 255, 255), thickness=1)
    true_mask = cv2.threshold(image, 254, 255, cv2.THRESH_BINARY)[1]


    t_min = threshold[0]
    t_max = threshold[1]

    th = cv2.threshold(slice[layer, : ,:].copy(), t_min, t_max, cv2.THRESH_BINARY_INV)[1]
    masked_th = cv2.bitwise_and(th,th,mask = box)

    gauss = cv2.GaussianBlur(slice[layer, : ,:].copy(), (5, 5), 0)
    thresh_otsu = cv2.threshold(gauss,0, 255,cv2.THRESH_BINARY_INV+cv2.THRESH_OTSU)[1]
    masked_otsu = cv2.bitwise_and(thresh_otsu , thresh_otsu, mask = box)


    fig, ax = plt.subplots(1, 3, figsize=(12, 8), constrained_layout=True)

    ax[0].imshow(true_mask, cmap="gray")
    ax[0].axis("off")
    ax[0].set_title("True mask")


    ax[1].imshow(masked_th, cmap="gray")
    ax[1].axis("off")
    ax[1].set_title(f"Global Thresh {t_min, t_max}")

    ax[2].imshow(masked_otsu, cmap="gray")
    ax[2].axis("off")
    ax[2].set_title("Otsu")
    

### Comparing masks

In [ ]:
@interact(layer=(positions[0], positions[-1]), threshold=IntRangeSlider(value=[63,120], min=0, max=255, step=1, description='Global Threshold:') )
def comparing_masks(layer, threshold):

    image = slice[layer, : ,:].copy()
    box = np.zeros(image.shape[:2], np.uint8)
    box[150:350, 150:350] = 255


    pts = np.array([(x, y) for(x, y) in zip(xs[layer - positions[0]], ys[layer - positions[0]])])

    cv2.drawContours(image, [pts], -1, (255, 255, 255), -1)
    cv2.polylines(image, [pts], isClosed=True, color=(255, 255, 255), thickness=1)
    true_mask = cv2.threshold(image, 254, 255, cv2.THRESH_BINARY)[1]


    t_min = threshold[0]
    t_max = threshold[1]

    th = cv2.threshold(slice[layer, : ,:].copy(), t_min, t_max, cv2.THRESH_BINARY_INV)[1]
    masked_th = cv2.bitwise_and(th,th,mask = box)

    gauss = cv2.GaussianBlur(slice[layer, : ,:].copy(), (5, 5), 0)
    thresh_otsu = cv2.threshold(gauss,0, 255,cv2.THRESH_BINARY_INV+cv2.THRESH_OTSU)[1]
    masked_otsu = cv2.bitwise_and(thresh_otsu , thresh_otsu, mask = box)


    fig, ax = plt.subplots(1, 3, figsize=(12, 8), constrained_layout=True)

    ax[0].imshow(true_mask, cmap="gray")
    ax[0].axis("off")
    ax[0].set_title("True mask")


    ax[1].imshow(masked_th, cmap="gray")
    ax[1].axis("off")
    ax[1].set_title(f"Global Thresh {t_min, t_max}")

    ax[2].imshow(masked_otsu, cmap="gray")
    ax[2].axis("off")
    ax[2].set_title("Otsu")

### Gabor filters

In [ ]:

ksize = 3
sigma = 1
theta = 2*np.pi/4
lamb = 2*np.pi/4
gamma = 0.5
phi = 0

kernel = cv2.getGaborKernel((ksize, ksize), sigma, theta, lamb, gamma, phi, ktype=cv2.CV_32F)

layer = 16
img = slice[layer, :, :].copy()

filtered_img = cv2.filter2D(img, cv2.CV_8UC3, kernel)

fig, ax = plt.subplots(1, 2, figsize=(12, 8 ), constrained_layout=True)

ax[0].imshow(kernel)
ax[0].set_title("Kernel")

ax[1].imshow(filtered_img, cmap="gray")
ax[1].set_title("Filtered image")


# Machine Learning Approach

## Importing modules for ML 

In [ ]:
import pandas as pd
from sklearn.model_selection import train_test_split
from sklearn.ensemble import RandomForestClassifier
from sklearn import metrics


## Feature extraction

### Labels

In [ ]:
def make_labels(slice, layer):

    if not layer in positions:
        print("no labels found!")
        

    else:
        #create layers from given ROI points
        image = slice[layer, :, :].copy()

        
        pts = np.array([(x, y) for(x, y) in zip(xs[layer - positions[0]], ys[layer - positions[0]])])

        cv2.drawContours(image, [pts], -1, (255, 255, 255), -1)
        cv2.polylines(image, [pts], isClosed=True, color=(255, 255, 255), thickness=1)
        label = cv2.threshold(image, 254, 255, cv2.THRESH_BINARY)[1]

        return label

In [ ]:
def features_extraction(slice, layer, ksize, Train_mode=False):

    df = pd.DataFrame() # create dataframe for features

    img = slice[layer, :, :].copy()

    img_vect = img.reshape(-1) # reshape 2D image into 1D vector
    df['Original pixels'] = img_vect 


    #generate a bunch of Gabor filters  
    i = 1 
    for theta in np.arange(0, np.pi, np.pi/4):
        for sigma in np.arange(1, 6, 1):
            for lamb in np.arange(0, np.pi, np.pi/4):
                for gamma in np.arange(0,1, 0.5):
                    for phi in np.arange(0,1, 0.5):
                        kernel = cv2.getGaborKernel((ksize, ksize), sigma, theta, lamb, gamma, phi, ktype=cv2.CV_32F)

                        fimg = cv2.filter2D(img, cv2.CV_8UC3, kernel)
                        
                        fimg_vect = fimg.reshape(-1)

                        Gabor_label = "Gabor" + str(i)
                        df[Gabor_label] = fimg_vect

                        i += 1



    #roberts
    roberts = skimage.filters.roberts(img)
    roberts_vect = roberts.reshape(-1)
    df['roberts'] = roberts_vect

    #sobel
    sobel = skimage.filters.sobel(img)
    sobel_vect = sobel.reshape(-1)
    df['sobel'] = sobel_vect

    #scharr
    scharr = skimage.filters.scharr(img)
    scharr_vect = scharr.reshape(-1)
    df['scharr'] = scharr_vect

    #prewitt
    prewitt = skimage.filters.prewitt(img)
    prewitt_vect = prewitt.reshape(-1)
    df['prewitt'] = prewitt_vect

    #canny
    canny = skimage.feature.canny(img, sigma=3)
    canny_vect = canny.reshape(-1)
    df['canny'] = canny_vect


    #gauss
    gauss = cv2.GaussianBlur(img, (ksize, ksize), 0)
    gauss_vect = gauss.reshape(-1)
    df['gauss'] = gauss_vect

    #otsu
    otsu = cv2.threshold(gauss,0, 255,cv2.THRESH_BINARY_INV+cv2.THRESH_OTSU)[1]
    otsu_vect = otsu.reshape(-1)
    df['otsu'] = otsu_vect



                        
    if Train_mode:

        labels = make_labels(slice, layer)
        labels_vect = labels.reshape(-1)
        df['labels'] = labels_vect

    return df


df = features_extraction(slice=slice, layer=14, ksize=5, Train_mode=True)


## Defining Variables

In [ ]:
Y = df['labels'].values
X = df.drop(labels= ['labels'], axis=1)

X_train, X_test, Y_train, Y_test = train_test_split(X, Y, test_size=0.25, random_state=20)

## Random Forest Classifier

In [ ]:
model = RandomForestClassifier(n_estimators=100, random_state=42)

model.fit(X_train, Y_train)

prediction_test = model.predict(X_test)
print("Accuracy = ", metrics.accuracy_score(Y_test, prediction_test))


### Most relevant filters

In [ ]:

features_list = list(X.columns)
features_imp = pd.Series(model.feature_importances_, index=features_list).sort_values(ascending=False)

pd.options.display.float_format = '{:,.10f}'.format
print(features_imp)

## Pickling model

In [ ]:
import pickle

filename = 'RF_model'
pickle.dump(model, open(filename, 'wb')) # wb - write binary 

## Test model predictions 

### Loading model

In [ ]:
load_model = pickle.load(open(filename, 'rb')) # rb - read binary 

### Test #1

In [ ]:
result = load_model.predict(X)

segmented = result.reshape(img.shape)

fig, ax = plt.subplots(1, 2, figsize=(12,8), constrained_layout=True)

ax[0].imshow(segmented, cmap="gray")
ax[0].axis("off")
ax[0].set_title("predicted")

label_img = make_labels(slice=slice, layer=14)

ax[1].imshow(label_img, cmap="gray")
ax[1].axis("off")
ax[1].set_title("ground truth")


# 3D Exploration

## Importing modules

In [ ]:
import ipyvolume as ipv
import scipy.ndimage
from plotly.figure_factory import create_trisurf
from plotly.offline import download_plotlyjs, init_notebook_mode, plot, iplot
from mpl_toolkits.mplot3d.art3d import Poly3DCollection

## ipyvolume

In [ ]:

mri_vol = ipv.volshow(
    slice, lighting=True,
    data_min=0, data_max=255, stereo=True)

ipv.show()



##  3D mesh 

In [ ]:


def resample(image, dcmfiles, new_spacing=[1, 1, 1]):
    # Determine current pixel spacing

    slice_thick = float(pydicom.read_file(dcmfiles[0], force=True).get(
        "SliceThickness"))

    pix_space = pydicom.read_file(dcmfiles[0], force=True).get(
        "PixelSpacing")

    pix_space = [float(x) for x in pix_space]

    spacing = map(float, ([slice_thick] + pix_space))
    spacing = np.array(list(spacing))

    resize_factor = spacing / new_spacing
    new_real_shape = image.shape * resize_factor
    new_shape = np.round(new_real_shape)
    real_resize_factor = new_shape / image.shape
    new_spacing = spacing / real_resize_factor

    image = scipy.ndimage.interpolation.zoom(image, real_resize_factor)

    return image, new_spacing


print("Shape before resampling\t", slice.shape)
imgs_after_resamp, spacing = resample(slice, files, [1, 1, 1])
print("Shape after resampling\t", imgs_after_resamp.shape)



In [ ]:


def make_mesh(image, step_size=1):

    print("Transposing surface")
    p = image.transpose(2, 1, 0)

    print("Calculating surface")
    verts, faces, norm, val = measure.marching_cubes(
        p, step_size=step_size, allow_degenerate=True)
    return verts, faces


def plotly_3d(verts, faces):
    x, y, z = zip(*verts)

    print("Drawing")

    # Make the colormap single color since the axes are positional not intensity.

    colormap = ['rgb(0, 0, 0)', 'rgb(255, 255, 255)']

    fig = create_trisurf(x=x,
                         y=y,
                         z=z,
                         plot_edges=True,
                         simplices=faces,
                         colormap=colormap,
                         title="Interactive Visualization")
    iplot(fig)



### 3D mesh plotly

In [ ]:
v, f = make_mesh(imgs_after_resamp, 2)
plotly_3d(v, f)